#### Day-1

##### Bigram and Trigram - DTM(Document Term Matrix)

In [1]:
import numpy as np
import pandas as pd
import string
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk

In [2]:
nltk.download("stopwords")
nltk.download("vader_lexicon")

[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Access Denied>
[nltk_data] Error loading vader_lexicon: <urlopen error Tunnel
[nltk_data]     connection failed: 407 Proxy Access Denied>


False

In [3]:
data = pd.read_excel("D:/Term-1/EDA/Amazon.xlsx")
data.head()

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,keys,...,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,upc,weight
0,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I initially had trouble deciding between the p...,"Paperwhite voyage, no regrets!",NaN,NaN,Cristina M,NaN,NaN,205 grams
1,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,Allow me to preface this with a little history...,One Simply Could Not Ask For More,NaN,NaN,Ricky,NaN,NaN,205 grams
2,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,4.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I am enjoying it so far. Great for reading. Ha...,Great for those that just want an e-reader,NaN,NaN,Tedd Gardiner,NaN,NaN,205 grams
3,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I bought one of the first Paperwhites and have...,Love / Hate relationship,NaN,NaN,Dougal,NaN,NaN,205 grams
4,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I have to say upfront - I don't like coroporat...,I LOVE IT,NaN,NaN,Miljan David Tanic,NaN,NaN,205 grams


In [4]:
text=data['reviews.text']

In [5]:
text[0]

"I initially had trouble deciding between the paperwhite and the voyage because reviews more or less said the same thing: the paperwhite is great, but if you have spending money, go for the voyage.Fortunately, I had friends who owned each, so I ended up buying the paperwhite on this basis: both models now have 300 ppi, so the 80 dollar jump turns out pricey the voyage's page press isn't always sensitive, and if you are fine with a specific setting, you don't need auto light adjustment).It's been a week and I am loving my paperwhite, no regrets! The touch screen is receptive and easy to use, and I keep the light at a specific setting regardless of the time of day. (In any case, it's not hard to change the setting either, as you'll only be changing the light level at a certain time of day, not every now and then while reading).Also glad that I went for the international shipping option with Amazon. Extra expense, but delivery was on time, with tracking, and I didnt need to worry about cu

In [6]:
data.shape # no of columns in the data frame is equal to no of columns in the DTM 

(1597, 27)

In [7]:
data['Clean_text'] = data['reviews.text'].str.lower().str.replace('[^a-z ]', ' ')

In [8]:
text = data[["reviews.text","Clean_text"]]

In [9]:
text.head()

,reviews.text,Clean_text
0,I initially had trouble deciding between the p...,i initially had trouble deciding between the p...
1,Allow me to preface this with a little history...,allow me to preface this with a little history...
2,I am enjoying it so far. Great for reading. Ha...,i am enjoying it so far great for reading ha...
3,I bought one of the first Paperwhites and have...,i bought one of the first paperwhites and have...
4,I have to say upfront - I don't like coroporat...,i have to say upfront i don t like coroporat...


In [10]:
# Removing the stop words

stop = set(stopwords.words('english'))

def sw(text):
    text = [word for word in text.split() if word not in stop] # spliting the words with space
    return " ".join(text) #

text["Split_words"] = text['Clean_text'].apply(sw)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
# Creating the count Vectorizer 

bigram_count_vect = CountVectorizer(ngram_range=(2,2))
# min combination of words is 2 and max combination of words is 4
#,min_df=0.01,max_df=0.5
#Fitting the count Vectorizer on slit words

bigram_count_vect.fit(text['Split_words']) #fitting the vectorizor on the split words 


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(2, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [12]:
# Creating a DTM matrix 

DTM = bigram_count_vect.fit_transform(text['Split_words'])
DTM

<1597x42941 sparse matrix of type '<class 'numpy.int64'>'
	with 125945 stored elements in Compressed Sparse Row format>

In [13]:
DTM_bigram = pd.DataFrame(DTM.toarray(),columns=bigram_count_vect.get_feature_names())
# get the COlUMN NAMES in the DTM matrix we need to use columns=bigram_count_vect.get_feature_names
DTM_bigram.head()

,aa batteries,abc live,abilities amd,abilities one,ability check,ability choose,ability connect,ability control,ability create,ability cut,...,zero eye,zero look,zink happy,zip code,zippy awesome,zombies love,zoom work,zooming supported,zumi blues,zw youll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# frequency table of DTM
word_count_freq = DTM_bigram.sum()
word_count_freq

#Converting the frequency table into DataFrame

word_table_bigram = pd.DataFrame(word_count_freq).reset_index()
word_table_bigram = word_table_bigram.rename(columns = {'index':"Word",
                                                      0:"Count"})
word_table_bigram.sort_values(by ="Count",ascending = False).head()

,Word,Count
19072,kindle fire,487
13289,fire hd,373
13381,fire tv,296
2330,apple buds,236
19594,last year,235


TF(w) = (Number of times term w appears is a document)/(Total number of terms in the document )

* it tells how popular is a word in the document 
* TF is known as simple term frequency

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)
* it tells how popular is a word in all the documents 

=================================================
                                       * Tarun        Manipal
- D1 ----> My name is Tarun            * 1/4            0
- D2 ----> tarun studies in Manipal    * 1/4            1
- D3 ----> tarun is a Data Scientest   * 1/4            0
- D4 ----> my age is 22                *  0             0 

* tarun is most frequently occuring in the data and it has no importent 
* Manipal is very importent word than the word tarun as it is has the context based importance 
* very frequenty occuring words will have very less TFIDF is high and vise versa

#### TFIDF Vectorizer

In [15]:
# Creating the TFIDF Vectorizer 

bigram_tfidf_vect = TfidfVectorizer(ngram_range=(2,2))
# min combination of words is 2 and max combination of words is 4
#,min_df=0.01,max_df=0.5
#Fitting the count Vectorizer on slit words

bigram_tfidf_vect.fit(text['Split_words']) #fitting the vectorizor on the split words 


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [16]:
z = bigram_tfidf_vect.fit_transform(text['Split_words'])
z

<1597x42941 sparse matrix of type '<class 'numpy.float64'>'
	with 125945 stored elements in Compressed Sparse Row format>

In [17]:
TFIDF_bigram = pd.DataFrame(z.toarray(),columns=bigram_tfidf_vect.get_feature_names())
# get the COlUMN NAMES in the DTM matrix we need to use columns=bigram_count_vect.get_feature_names
TFIDF_bigram.head()

,aa batteries,abc live,abilities amd,abilities one,ability check,ability choose,ability connect,ability control,ability create,ability cut,...,zero eye,zero look,zink happy,zip code,zippy awesome,zombies love,zoom work,zooming supported,zumi blues,zw youll
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# frequency table of DTM
word_tfidf_freq = TFIDF_bigram.sum()
word_tfidf_freq

#Converting the frequency table into DataFrame

word_table_bigram_tfidf = pd.DataFrame(word_tfidf_freq).reset_index()
word_table_bigram_tfidf = word_table_bigram_tfidf.rename(columns = {'index':"Word",
                                                      0:"Score"})
word_table_bigram_tfidf.sort_values(by ="Score",ascending = False).head()

,Word,Score
19072,kindle fire,20.891025
2330,apple buds,18.391322
13289,fire hd,15.798090
13381,fire tv,12.918418
19594,last year,11.459391


===============================================================================

In [19]:
#document1 = '''Python is a 2000 made-for-TV horror movie directed by Richard Clabaugh. The film features several cult favorite actors, including William Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy, Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean Whalen. The film concerns a genetically engineered snake, a python, that escapes and unleashes itself on a small town. It includes the classic final girl scenario evident in films like Friday the 13th. It was filmed in Los Angeles,  California and Malibu, California. Python was followed by two sequels: Python  II (2002) and Boa vs. Python (2004), both also made-for-TV films.'''
#document2 = '''Python, from the Greek word (πύθων/πύθωνας), is a genus of nonvenomous pythons found in Africa and Asia. Currently, 7 species are recognised. A member of this genus, P. reticulatus, is among the longest snakes known.'''
document3 = '''The Colt Python is a .357 Magnum caliber revolver formerly manufactured by Colt's Manufacturing Company of Hartford, Connecticut. It is sometimes referred to as a "Combat Magnum". It was first introduced in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued Colt Python targeted the premium revolver market segment. Some firearm collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy Thompson, Renee Smeets and Martin Dougherty have described the Python as the finest production revolver ever made.'''


In [20]:
import pandas as pd

In [21]:
review=[document3]
count=[1,]
new_df=pd.DataFrame({'count': count,'review':review})

In [22]:
new_df

,count,review
0,1,The Colt Python is a .357 Magnum caliber revol...


In [23]:
new_df['Clean_text'] = new_df['review'].str.lower().str.replace('[^a-z ]', ' ')

In [24]:
new_df.head()

,count,review,Clean_text
0,1,The Colt Python is a .357 Magnum caliber revol...,the colt python is a magnum caliber revol...


In [25]:
new_df['Clean_text'][0]

'the colt python is a      magnum caliber revolver formerly manufactured by colt s manufacturing company of hartford  connecticut  it is sometimes referred to as a  combat magnum   it was first introduced in       the same year as smith  amp  wesson s m       magnum  the now discontinued colt python targeted the premium revolver market segment  some firearm collectors and writers such as jeff cooper  ian v  hogg  chuck hawks  leroy thompson  renee smeets and martin dougherty have described the python as the finest production revolver ever made '

In [26]:
# Removing the stop words

stop = set(stopwords.words('english'))

def sw(text):
    text = [word for word in text.split() if word not in stop] # spliting the words with space
    return " ".join(text) #

new_df["Split_words"] = new_df['Clean_text'].apply(sw)

In [32]:
new_df

,count,review,Clean_text,Split_words
0,1,The Colt Python is a .357 Magnum caliber revol...,the colt python is a magnum caliber revol...,colt python magnum caliber revolver formerly m...


In [27]:
# Creating the TFIDF Vectorizer 

bigram_tfidf_vect = TfidfVectorizer(ngram_range=(1,1))
# min combination of words is 2 and max combination of words is 4
#,min_df=0.01,max_df=0.5
#Fitting the count Vectorizer on slit words

bigram_tfidf_vect.fit(new_df['Split_words']) #fitting the vectorizor on the split words 


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [28]:
k = bigram_tfidf_vect.fit_transform(new_df['Split_words'])
k

<1x45 sparse matrix of type '<class 'numpy.float64'>'
	with 45 stored elements in Compressed Sparse Row format>

In [29]:
TFIDF_bigram = pd.DataFrame(k.toarray(),columns=bigram_tfidf_vect.get_feature_names())
# get the COlUMN NAMES in the DTM matrix we need to use columns=bigram_count_vect.get_feature_names
TFIDF_bigram.head()

,amp,caliber,chuck,collectors,colt,combat,company,connecticut,cooper,described,...,revolver,segment,smeets,smith,sometimes,targeted,thompson,wesson,writers,year
0,0.113961,0.113961,0.113961,0.113961,0.341882,0.113961,0.113961,0.113961,0.113961,0.113961,...,0.341882,0.113961,0.113961,0.113961,0.113961,0.113961,0.113961,0.113961,0.113961,0.113961


In [30]:
# frequency table of DTM
word_tfidf_freq = TFIDF_bigram.sum()
word_tfidf_freq
#Converting the frequency table into DataFrame

word_table_bigram_tfidf = pd.DataFrame(word_tfidf_freq).reset_index()
word_table_bigram_tfidf = word_table_bigram_tfidf.rename(columns = {'index':"Word",
                                                      0:"Score"})
word_table_bigram_tfidf.sort_values(by ="Score",ascending = False).head()


,Word,Score
35,revolver,0.341882
25,magnum,0.341882
4,colt,0.341882
32,python,0.341882
34,renee,0.113961
